In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import re
import json
import h2o

from datetime import datetime

from utils import get_data, Config, clean_url, make_clickable, add_columns

from tree import make_tree

In [3]:
# Load all data (except unknown/manual testing)
# Desktop Runs+Re-Runs were performed between 2024-02-05 and 2024-02-12
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown' AND "Result".created_at < '2024-02-13';
"""
df = get_data(Config(), initial_data)
df = add_columns(df)

Connecting to the PostgreSQL database...
Connection successful


In [4]:
df["raw_header"] = df["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str)

In [5]:
responses = """
SELECT * from "Response";
"""
responses = get_data(Config(), responses)

Connecting to the PostgreSQL database...
Connection successful


# Overview

In [6]:
# In total 2 runs + couple of "repeat"-mode runs such that all browsers have results for each test
# Safari has more results as we had to abort and restart the main run a couple of times
# Both basic + parsing mode
df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"])["id"].count().sort_values(ascending=False).to_frame().reset_index()

,automation_mode,browser_id,name,version,os,headless_mode,id
0,selenium,46,safari,17.3,macOS 14.3,real,396825
1,selenium,43,firefox,122,Ubuntu 22.04,headless,358476
2,selenium,42,chrome,121,Ubuntu 22.04,headless-new,358460
3,selenium,45,brave,v1.62.156 (121.0.6167.139),Ubuntu 22.04,headless-new,358337
4,selenium,44,edge,121,Ubuntu 22.04,headless-new,353368


# Error + Timeout Analysis
- Only timeout exist, no other systematic failures
- Systematic timeout: TAO in Safari -> fixed
- Couple of random timeouts in other tests

In [7]:
# Timeout (2), Fail (1), Not-run (3)
filtered_df = df.groupby(["browser"])["test_status"].filter(lambda x: x.nunique() >= 2)
display(df[["browser", "test_status"]].loc[df.index.isin(filtered_df.index)].groupby(["browser"])["test_status"].value_counts().to_frame())

count
browser                                            test_status        
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... 0            358288
                                                   2                49
chrome Ubuntu 22.04 121 selenium headless-new      0            358434
                                                   2                26
edge Ubuntu 22.04 121 selenium headless-new        0            353302
                                                   2                66
firefox Ubuntu 22.04 122 selenium headless         0            358466
                                                   2                10
safari macOS 14.3 17.3 selenium real               0            396664
                                                   2               161

In [8]:
# Teststatus == 2 (timeout)

# Rare timeouts in all browsers; Safari some more than others
display(df[df["test_status"] == 2]["browser"].value_counts().to_frame())

# Mostly 302 redirects for basic (oac_iframe, oac_window.open), referrer_iframe 
# perfAPI_img (only Safari, fixed!)
# Others quite rare
display(df.loc[(df["test_status"] == 2)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
safari macOS 14.3 17.3 selenium real,161
edge Ubuntu 22.04 121 selenium headless-new,66
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,49
chrome Ubuntu 22.04 121 selenium headless-new,26
firefox Ubuntu 22.04 122 selenium headless,10


count
resp_type status_code test_name           browser                                                  
basic     302         oac_iframe          safari macOS 14.3 17.3 selenium real                   40
          200         referrer_iframe     safari macOS 14.3 17.3 selenium real                   38
parsing   200         perfAPI_img         safari macOS 14.3 17.3 selenium real                   32
basic     302         referrer_iframe     safari macOS 14.3 17.3 selenium real                   26
                      oac_window.open     safari macOS 14.3 17.3 selenium real                   24
                      oac_iframe          brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     20
                      oac_window.open     edge Ubuntu 22.04 121 selenium headless-new            16
                                          brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     13
                      oac_iframe          edge Ubuntu 22.04 121 selenium headless-new            12
parsing   200         fullscreen_iframe   edge Ubuntu 22.04 121 selenium headless-new            10
basic     200         referrer_iframe     firefox Ubuntu 22.04 122 selenium headless              9
          302         referrer_iframe     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      9
          200         oac_iframe          edge Ubuntu 22.04 121 selenium headless-new             8
                      framing_embed       chrome Ubuntu 22.04 121 selenium headless-new           8
                      framing_object      chrome Ubuntu 22.04 121 selenium headless-new           8
          302         referrer_iframe     edge Ubuntu 22.04 121 selenium headless-new             7
          200         framing_iframe      chrome Ubuntu 22.04 121 selenium headless-new           7
                      referrer_iframe     edge Ubuntu 22.04 121 selenium headless-new             6
parsing   200         framing_iframe      edge Ubuntu 22.04 121 selenium headless-new             5
                                          brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      5
basic     200         referrer_iframe     chrome Ubuntu 22.04 121 selenium headless-new           1
                                          brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
parsing   200         accesswindow_direct edge Ubuntu 22.04 121 selenium headless-new             1
                                          firefox Ubuntu 22.04 122 selenium headless              1
                                          chrome Ubuntu 22.04 121 selenium headless-new           1
                      framing_iframe      safari macOS 14.3 17.3 selenium real                    1
                      fullscreen_iframe   chrome Ubuntu 22.04 121 selenium headless-new           1
                      oac_window.open     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                      referrer_iframe     edge Ubuntu 22.04 121 selenium headless-new             1

# General Test Statistics

In [9]:
df[["browser", "resp_type", "test_name", "relation_info", "response_id", "org_scheme", "org_host", "resp_scheme", "resp_host"]].drop_duplicates().groupby(["resp_type", "browser"]).count()

test_name  \
resp_type browser                                                         
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      10456   
          chrome Ubuntu 22.04 121 selenium headless-new           10456   
          edge Ubuntu 22.04 121 selenium headless-new             10456   
          firefox Ubuntu 22.04 122 selenium headless              10456   
          safari macOS 14.3 17.3 selenium real                    10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     168774   
          chrome Ubuntu 22.04 121 selenium headless-new          168774   
          edge Ubuntu 22.04 121 selenium headless-new            168774   
          firefox Ubuntu 22.04 122 selenium headless             168774   
          safari macOS 14.3 17.3 selenium real                   168774   

                                                              relation_info  \
resp_type browser                                                             
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...          10456   
          chrome Ubuntu 22.04 121 selenium headless-new               10456   
          edge Ubuntu 22.04 121 selenium headless-new                 10456   
          firefox Ubuntu 22.04 122 selenium headless                  10456   
          safari macOS 14.3 17.3 selenium real                        10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...         168774   
          chrome Ubuntu 22.04 121 selenium headless-new              168774   
          edge Ubuntu 22.04 121 selenium headless-new                168774   
          firefox Ubuntu 22.04 122 selenium headless                 168774   
          safari macOS 14.3 17.3 selenium real                       168774   

                                                              response_id  \
resp_type browser                                                           
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...        10456   
          chrome Ubuntu 22.04 121 selenium headless-new             10456   
          edge Ubuntu 22.04 121 selenium headless-new               10456   
          firefox Ubuntu 22.04 122 selenium headless                10456   
          safari macOS 14.3 17.3 selenium real                      10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...       168774   
          chrome Ubuntu 22.04 121 selenium headless-new            168774   
          edge Ubuntu 22.04 121 selenium headless-new              168774   
          firefox Ubuntu 22.04 122 selenium headless               168774   
          safari macOS 14.3 17.3 selenium real                     168774   

                                                              org_scheme  \
resp_type browser                                                          
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...       10456   
          chrome Ubuntu 22.04 121 selenium headless-new            10456   
          edge Ubuntu 22.04 121 selenium headless-new              10456   
          firefox Ubuntu 22.04 122 selenium headless               10456   
          safari macOS 14.3 17.3 selenium real                     10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      168774   
          chrome Ubuntu 22.04 121 selenium headless-new           168774   
          edge Ubuntu 22.04 121 selenium headless-new             168774   
          firefox Ubuntu 22.04 122 selenium headless              168774   
          safari macOS 14.3 17.3 selenium real                    168774   

                                                              org_host  \
resp_type browser                                                        
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     10456   
          chrome Ubuntu 22.04 121 selenium headless-new          10456   
          edge Ubuntu 22.04 121 selenium headless-new            10456   
    

In [10]:
df.groupby("browser")["clean_url"].nunique()

browser
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    85492
chrome Ubuntu 22.04 121 selenium headless-new                          85492
edge Ubuntu 22.04 121 selenium headless-new                            85492
firefox Ubuntu 22.04 122 selenium headless                             85492
safari macOS 14.3 17.3 selenium real                                   85492
Name: clean_url, dtype: int64

In [11]:
df.groupby("browser")["response_id"].nunique()

browser
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    44415
chrome Ubuntu 22.04 121 selenium headless-new                          44415
edge Ubuntu 22.04 121 selenium headless-new                            44415
firefox Ubuntu 22.04 122 selenium headless                             44415
safari macOS 14.3 17.3 selenium real                                   44415
Name: response_id, dtype: int64

In [12]:
# Make sure that all responses actually have responses
# (create_repeat.py assumes that at least one browser collected the results)

r1 = df.groupby("label")["response_id"].nunique().sort_values()
r2 = responses.loc[responses["resp_type"] != "debug"].groupby("label")["id"].nunique().sort_values()

pd.concat([r1, r2], axis=1)

,response_id,id
label,,
CORS,6,6
CORS-ACAC,864,864
CSPvsXFO,1447,1447
CORS-ACAM,1576,1576
CORS-ACEH,1577,1577
CORS-ACAH,1577,1577
TAO,1614,1614
OAC,1713,1713
CORS-ACAO,2465,2465


In [13]:
df.groupby(["resp_type", "test_name", "relation_info"])["response_id"].nunique().to_frame()

response_id
resp_type test_name                     relation_info              
basic     accesswindow_direct           direct                   11
          fetch_GET                     credentials               6
                                        custom_headers            6
                                        simple                    6
          fetch_TEST                    custom_method             6
          framing_embed                 direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_iframe                direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_object                direct                   35
                                        nested                   35
                                        sandbox                  35
          fullscreen_iframe             child                    24
                                        child_allow              24
                                        child_sandbox            24
                                        direct                   24
          imgloading_iframe             direct                   17
          oac_iframe                    direct                   14
                                        sandbox                  14
          oac_window.open               window.open              14
          perfAPI_img                   direct                   13
          referrer_iframe               iframe                   16
                                        window.open              16
          script_execution_iframe       direct                   17
                                        sandbox                  17
          subresourceloadingCOEP_img    direct                   11
                                        sandbox                  11
          subresourceloadingCORP_img    direct                   11
                                        nested                   11
                                        sandbox                  11
          subresourceloadingCORP_object direct                   11
          upgradeHSTS_direct            direct                   14
          upgradeHSTS_subdomain         subdomain                14
parsing   accesswindow_direct           direct                 2656
          fetch_GET                     credentials            8059
                                        custom_headers         8059
                                        simple                 8059
          fetch_TEST                    custom_method          8059
          framing_iframe                direct                 8780
                                        nested                 8780
          fullscreen_iframe             child_allow            3880
                                        direct                 3880
          imgloading_iframe             direct                 2710
          oac_window.open               window.open            1699
          perfAPI_img                   direct                 1601
          referrer_iframe               iframe                 2797
          script_execution_iframe       direct                 2726
          subresourceloadingCOEP_img    direct                 2637
          subresourceloadingCORP_img    direct                 3357
          upgradeHSTS_direct            direct                 3324
          upgradeHSTS_subdomain         subdomain              3324

# Stability
- Basic tests:
    - 21/(10456x5) tests with different outcomes!
    - 13x firefox subresourceloadingCOEP_img (random behavior bug)
    - 6x timeout vs message 4(x edge, 2x brave)
    - 1x brave referrer different outcome
    - 1x safari COEP different outcome?
- Parsing tests:
    - 755/(168774x5) tests with different outcomes!
    - quite a lot! last time we only had 1 single test with a different outcome? (We used different versions of the browsers)
    - 3x accesswindow_direct (null vs accessible), 1x fullscreen_iframe (timeout vs message), 1x img_loading_iframe (timeout vs message)
    - All others (700+) upgradeHSTS_subdomain and upgradeHSTS_direct
        - Prevalence: `brave ~= edge > chrome >> firefox >> safari`
        - Hints at systematic issue: maybe the reset did not work or something similar?
        - Mostly timeout vs redirect=false; however some are also timeout vs redirect=false vs redirect=true!
        - Timeout maybe related to the HTTPS upgrade feature in chromium-based browsers which can take some time (>5s?)? (Non-Authoritative-Reason: HttpsUpgrades)
        - Unclear how redirect true vs false could happen? Maybe HSTS clearing did not work?
        - Manual testing always results in redirect=false for all tried clean_urls!

In [14]:
# Remove all tests that timed out (test_status == 0), each test has at least one result that did not time out in each browser
df_org = df
df = df.loc[df["test_status"] == 0]

In [15]:
for group_name, resp_type_group in df.groupby("resp_type"):
    if group_name == "basic":
        continue
    print(group_name)
    stab = resp_type_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])[["outcome_str"]].agg(["count", "nunique"])
    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["browser"].value_counts().to_frame())
    display(diff_outcomes.reset_index()[["test_name", "browser"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "browser"])["response_id"].agg(list).to_frame().reset_index()
    
    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        # TODO: shortcut, only show the first result for each test_name/browser; results are usually always the same!
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["browser"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "name"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)
        # input("Continue!")

parsing
755 tests have different outcomes!


,count
browser,
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,300
edge Ubuntu 22.04 121 selenium headless-new,282
chrome Ubuntu 22.04 121 selenium headless-new,133
firefox Ubuntu 22.04 122 selenium headless,34
safari macOS 14.3 17.3 selenium real,6


count
(test_name, )         (browser, )                                              
upgradeHSTS_subdomain edge Ubuntu 22.04 121 selenium headless-new           191
                      brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...    171
upgradeHSTS_direct    brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...    128
                      edge Ubuntu 22.04 121 selenium headless-new            89
upgradeHSTS_subdomain chrome Ubuntu 22.04 121 selenium headless-new          82
upgradeHSTS_direct    chrome Ubuntu 22.04 121 selenium headless-new          50
                      firefox Ubuntu 22.04 122 selenium headless             25
upgradeHSTS_subdomain firefox Ubuntu 22.04 122 selenium headless              9
upgradeHSTS_direct    safari macOS 14.3 17.3 selenium real                    3
accesswindow_direct   edge Ubuntu 22.04 121 selenium headless-new             2
upgradeHSTS_subdomain safari macOS 14.3 17.3 selenium real                    2
fullscreen_iframe     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
accesswindow_direct   chrome Ubuntu 22.04 121 selenium headless-new           1
imgloading_iframe     safari macOS 14.3 17.3 selenium real                    1

accesswindow_direct [22059, 22478]


,,count
outcome_str,name,
{'window.open.opener': 'null'},edge,2
"{'window.open.opener': 'object ""[object Window]""'}",edge,2


...

,,count
outcome_str,name,
Fetch timed out,firefox,9
{'response.redirected': False},firefox,9


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
187082,firefox Ubuntu 22.04 122 selenium headless,{'response.redirected': False},"[['strict-transport-security', 'max-a?ge=60']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=32149&last_id=32149&scheme=http
228017,firefox Ubuntu 22.04 122 selenium headless,Fetch timed out,"[['strict-transport-security', 'max-a?ge=60']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=32149&last_id=32149&scheme=http


upgradeHSTS_subdomain [29963, 33015]


,,count
outcome_str,name,
Fetch timed out,safari,2
{'response.redirected': False},safari,2


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
259077,safari macOS 14.3 17.3 selenium real,{'response.redirected': False},"[['strict-transport-securityO', 'max-age=0']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=29963&last_id=29963&scheme=http
268353,safari macOS 14.3 17.3 selenium real,Fetch timed out,"[['strict-transport-securityO', 'max-age=0']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=29963&last_id=29963&scheme=http


# Browser differences!

In [16]:
df_stab = df

# Only keep one row for each test
# If the test had non-deterministic results, chose the first?
# TODO: for upgradeHSTS instead choose {"response.redirected": false}!

print("Original data entries", len(df))
df = df.drop_duplicates(subset=["test_name", "relation_info", "browser", "org_origin", "resp_origin", "response_id"])
print("Only one row for each test (per browser)", len(df))

Original data entries 1825154
Only one row for each test (per browser) 896150


In [17]:
# Most tests have exactly one outcome, quite some have two, couple have three!
res = df.groupby(["resp_type", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
res = res["outcome_str"].agg("nunique")
display(res.value_counts().to_frame())

,count
outcome_str,
1,170578
2,8576
3,76


In [18]:
# Basic:
# subresourceloadingCORP_object has different results for 100% of tests
# couple of others have 20%+ differences
# coupel has 0%

# Parsing:
# oac_window.open has 99%
# Other test groups have between 0.5 - 7.5% of tests with different outcomes
r = res.reset_index()
r["diff"] = r["outcome_str"] != 1
r.groupby(["resp_type", "test_name", "relation_info"])["diff"].agg(["count", "sum", "mean"]).sort_values("mean", ascending=False)

count   sum      mean
resp_type test_name                     relation_info                        
basic     subresourceloadingCORP_object direct            176   176  1.000000
parsing   oac_window.open               window.open      3398  3370  0.991760
basic     oac_window.open               window.open       224   192  0.857143
          oac_iframe                    direct            224   144  0.642857
                                        sandbox           224   144  0.642857
          referrer_iframe               window.open       256   117  0.457031
          fullscreen_iframe             direct            384   142  0.369792
          framing_embed                 direct            560   186  0.332143
          framing_object                direct            560   186  0.332143
          fullscreen_iframe             child_allow       384   113  0.294271
          perfAPI_img                   direct            208    55  0.264423
          subresourceloadingCOEP_img    direct            176    45  0.255682
          framing_object                sandbox           560   120  0.214286
                                        nested            560   120  0.214286
          framing_embed                 nested            560   120  0.214286
                                        sandbox           560   120  0.214286
          accesswindow_direct           direct            176    28  0.159091
          script_execution_iframe       sandbox           272    21  0.077206
parsing   upgradeHSTS_direct            direct           6648   490  0.073706
basic     referrer_iframe               iframe            256    18  0.070312
parsing   upgradeHSTS_subdomain         subdomain        6648   417  0.062726
basic     fullscreen_iframe             child             384    23  0.059896
          imgloading_iframe             direct            272    13  0.047794
parsing   fullscreen_iframe             child_allow      7760   356  0.045876
          imgloading_iframe             direct           5420   230  0.042435
          script_execution_iframe       direct           5452   216  0.039618
basic     upgradeHSTS_direct            direct             28     1  0.035714
          upgradeHSTS_subdomain         subdomain          28     1  0.035714
parsing   fullscreen_iframe             direct           7760   272  0.035052
          framing_iframe                direct          17560   365  0.020786
          referrer_iframe               iframe           5594   101  0.018055
basic     subresourceloadingCOEP_img    sandbox           176     3  0.017045
          framing_iframe                sandbox           560     8  0.014286
parsing   subresourceloadingCOEP_img    direct           5274    60  0.011377
          perfAPI_img                   direct           3202    33  0.010306
          subresourceloadingCORP_img    direct           6714    68  0.010128
          framing_iframe                nested          17560   144  0.008200
          accesswindow_direct           direct           5312    40  0.007530
          fetch_GET                     simple          16118   104  0.006452
          fetch_TEST                    custom_method   16118   101  0.006266
          fetch_GET                     custom_headers  16118   101  0.006266
basic     subresourceloadingCORP_img    direct            176     1  0.005682
parsing   fetch_GET                     credentials     16118    86  0.005336
basic     framing_iframe                nested            560     1  0.001786
          fetch_GET                     custom_headers     96     0  0.000000
          fullscreen_iframe             child_sandbox     384     0  0.000000
          framing_iframe                direct            560     0  0.000000
          fetch_GET                     simple             96     0  0.000000
          fetch_TEST                    custom_method      96     0  0.000000
          fetch_GET                     credentials        96     0  0.000000


In [19]:
# Select only test rows with more than one outcome
tests_with_mult_outcomes = r.loc[r["diff"]].set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
df_mult = df.loc[df[["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]].apply(tuple, axis=1).isin(tests_with_mult_outcomes.index)]

print(f"All test rows: {len(df)}, Rows with more than one outcome: {len(df_mult)}")

All test rows: 896150, Rows with more than one outcome: 43260


In [20]:
# Display difference groups
for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    df_list = []
    for _, one_test in group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]):
        browsers = one_test.groupby("outcome_str")["browser"].unique().apply(list).apply(sorted).to_frame()
        new = True
        for df_b in df_list:
            if df_b.equals(browsers):
                new = False
        if new:
            df_list.append(browsers)
    print(len(df_list))
    for df_b in df_list:
        with pd.option_context("display.max_colwidth", 1000):
            display(df_b)
    #input("Continue!")

('accesswindow_direct', 'direct')
5


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 122 selenium headless, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 122 selenium headless]"
"{'window.open.opener': 'object ""[object Window]""'}",[safari macOS 14.3 17.3 selenium real]


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, safari macOS 14.3 17.3 selenium real]"
"{'window.open.opener': 'object ""[object Window]""'}",[firefox Ubuntu 22.04 122 selenium headless]


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 122 selenium headless]"
"{'window.open.opener': 'object ""[object Window]""'}","[edge Ubuntu 22.04 121 selenium headless-new, safari macOS 14.3 17.3 selenium real]"


...

,browser
outcome_str,
Fetch timed out,"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, firefox Ubuntu 22.04 122 selenium headless]"
{'response.redirected': False},"[chrome Ubuntu 22.04 121 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
Fetch timed out,[firefox Ubuntu 22.04 122 selenium headless]
{'response.redirected': True},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
{'response.redirected': False},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
{'response.redirected': True},"[firefox Ubuntu 22.04 122 selenium headless, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
Fetch timed out,[safari macOS 14.3 17.3 selenium real]
{'response.redirected': False},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 122 selenium headless]"


,browser
outcome_str,
Fetch timed out,[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new]
{'response.redirected': False},"[chrome Ubuntu 22.04 121 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 122 selenium headless]"
{'response.redirected': True},[safari macOS 14.3 17.3 selenium real]


In [21]:
# Other approach to list the difference groups
def get_uniques(df):
    browsers = f"{df['name'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    
    unique_outcomes = {}
    group.groupby(["name"])["outcome_str"].value_counts(normalize=False).reset_index().groupby(["outcome_str", "count"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        display(d)
        print()

,['safari'],['firefox'],['edge'],"['brave', 'chrome']"
{'window.open.opener': 'null'},16,24,54,56
"{'window.open.opener': 'object ""[object Window]""'}",52,44,14,12


,"['brave', 'chrome', 'edge']",['safari'],['firefox']
"{'error': 'null', 'headers': 'content-length,'}",3.0,9.0,10
"{'error': 'null', 'headers': 'content-length,test,'}",19.0,31.0,20
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",64.0,46.0,16
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,NaN,40


,['firefox'],"['brave', 'chrome', 'edge', 'safari']","['brave', 'chrome', 'edge']",['safari'],"['firefox', 'safari']"
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",2.0,4.0,NaN,NaN,NaN
"{'error': 'null', 'headers': 'content-length,'}",14.0,NaN,4.0,10.0,NaN
"{'error': 'null', 'headers': 'content-length,test,'}",24.0,NaN,25.0,34.0,NaN
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",21.0,NaN,68.0,13.0,NaN
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,NaN,NaN,NaN,40.0


,"['brave', 'chrome', 'edge', 'safari']",['firefox'],"['brave', 'chrome', 'edge']",['safari']
"{'error': 'null', 'headers': ''}",3.0,NaN,NaN,NaN
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,location,server,test,'}",1.0,NaN,NaN,NaN
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",4.0,2.0,NaN,NaN
"{'error': 'null', 'headers': 'cache-control,content-type,'}",NaN,10.0,NaN,NaN
"{'error': 'null', 'headers': 'content-length,'}",NaN,14.0,4.0,10.0
"{'error': 'null', 'headers': 'content-length,test,'}",NaN,26.0,25.0,40.0
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,40.0,NaN,NaN
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",NaN,12.0,67.0,46.0


,['firefox'],"['brave', 'chrome', 'edge', 'safari']","['brave', 'chrome', 'edge']",['safari'],"['firefox', 'safari']"
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",2.0,4.0,NaN,NaN,NaN
"{'error': 'null', 'headers': 'content-length,'}",14.0,NaN,4.0,10.0,NaN
"{'error': 'null', 'headers': 'content-length,test,'}",24.0,NaN,25.0,34.0,NaN
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",21.0,NaN,68.0,13.0,NaN
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,NaN,NaN,NaN,40.0


...

,['safari'],"['brave', 'chrome', 'edge', 'firefox']"
"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",3.0,NaN
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",NaN,3.0


,['safari'],['firefox'],"['brave', 'chrome', 'edge']","['firefox', 'safari']"
error,17.0,29.0,52.0,NaN
timeout: no event fired,12.0,NaN,NaN,NaN
load,NaN,NaN,17.0,40.0


,"['brave', 'chrome', 'edge']",['firefox'],['safari']
error,44.0,76.0,NaN
load,132.0,100.0,NaN
timeout: no event fired,NaN,NaN,176.0


,['safari'],['firefox'],['chrome'],['edge'],['brave'],"['brave', 'chrome', 'edge']"
Fetch timed out,2.0,25,30.0,72.0,85.0,NaN
{'response.redirected': False},203.0,432,210.0,169.0,158.0,NaN
{'response.redirected': True},286.0,25,224.0,223.0,221.0,NaN
TypeError: Failed to fetch,NaN,9,NaN,NaN,NaN,27.0


,['safari'],['firefox'],['chrome'],['brave'],['edge'],"['brave', 'chrome', 'edge']"
Fetch timed out,1.0,20,56.0,119.0,143.0,NaN
{'response.redirected': False},342.0,370,287.0,225.0,206.0,NaN
{'response.redirected': True},75.0,19,48.0,47.0,42.0,NaN
TypeError: Failed to fetch,NaN,9,NaN,NaN,NaN,27.0


In [22]:
# Function to calculate absolute and percentage overlap
def calculate_overlap(df, col1, col2):
    df = df[[col1, col2]].dropna()
    df.columns.values[0] = 0
    df.columns.values[1] = 1
    absolute_overlap = (df[0] == df[1]).sum()
    unique_rows = len(df)
    diff_rows = unique_rows - absolute_overlap
    percentage_diff = (diff_rows / unique_rows) * 100
    return diff_rows, percentage_diff, unique_rows


def display_overlap(df, show_all=True, name="All"):
    print(name)
    sim_frame = df.set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])["outcome_str"].unstack().astype("str")
    sim_frame = sim_frame.reset_index().drop(columns=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
    # Create a dictionary to store results
    results = {}
    matrix = {}
    
    # Iterate through all pairs of columns
    for i, col1 in enumerate(sim_frame.columns):
        for col2 in sim_frame.columns:
            key = f"{col1}_vs_{col2}"
            results[key] = calculate_overlap(sim_frame, col1, col2)
            try:
                l = matrix[col1]
            except KeyError:
                l = {}
            # 0 for absolute count of differences, 1 for percentage
            l[col2] = results[key][0]
            matrix[col1] = l
            
    
    # Convert the results to a DataFrame for better visualization
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Absolute Diff', 'Percentage Diff', "Count"])
    
    # Display the results
    if show_all:
        display(results_df)
    
    # Display difference matrix
    with pd.option_context("display.max_columns", 28):
        if show_all:
            display(pd.DataFrame(matrix))
        else:
            display(pd.DataFrame(matrix).drop_duplicates().T)


display_overlap(df)

for name, group in df.groupby("test_name"):
    display_overlap(group, show_all=False, name=name)

All


,Absolute Diff,Percentage Diff,Count
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,0.000000,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_chrome Ubuntu 22.04 121 selenium headless-new,432,0.241031,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_edge Ubuntu 22.04 121 selenium headless-new,562,0.313564,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_firefox Ubuntu 22.04 122 selenium headless,7090,3.955811,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_safari macOS 14.3 17.3 selenium real,6795,3.791218,179230
chrome Ubuntu 22.04 121 selenium headless-new_vs_brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,432,0.241031,179230
chrome Ubuntu 22.04 121 selenium headless-new_vs_chrome Ubuntu 22.04 121 selenium headless-new,0,0.000000,179230
chrome Ubuntu 22.04 121 selenium headless-new_vs_edge Ubuntu 22.04 121 selenium headless-new,306,0.170730,179230
chrome Ubuntu 22.04 121 selenium headless-new_vs_firefox Ubuntu 22.04 122 selenium headless,7000,3.905596,179230
chrome Ubuntu 22.04 121 selenium headless-new_vs_safari macOS 14.3 17.3 selenium real,6721,3.749930,179230


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,432,562,7090,6795
chrome Ubuntu 22.04 121 selenium headless-new,432,0,306,7000,6721
edge Ubuntu 22.04 121 selenium headless-new,562,306,0,7135,6849
firefox Ubuntu 22.04 122 selenium headless,7090,7000,7135,0,2740
safari macOS 14.3 17.3 selenium real,6795,6721,6849,2740,0


accesswindow_direct


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,2,56,40
chrome Ubuntu 22.04 121 selenium headless-new,0,2,56,40
edge Ubuntu 22.04 121 selenium headless-new,2,0,58,38
firefox Ubuntu 22.04 122 selenium headless,56,58,0,40
safari macOS 14.3 17.3 selenium real,40,38,40,0


...

,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,32,176
chrome Ubuntu 22.04 121 selenium headless-new,0,32,176
edge Ubuntu 22.04 121 selenium headless-new,0,32,176
firefox Ubuntu 22.04 122 selenium headless,32,0,176
safari macOS 14.3 17.3 selenium real,176,176,0


upgradeHSTS_direct


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,113,157,336,169
chrome Ubuntu 22.04 121 selenium headless-new,113,0,102,283,114
edge Ubuntu 22.04 121 selenium headless-new,157,102,0,324,157
firefox Ubuntu 22.04 122 selenium headless,336,283,324,0,296
safari macOS 14.3 17.3 selenium real,169,114,157,296,0


upgradeHSTS_subdomain


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,173,258,192,173
chrome Ubuntu 22.04 121 selenium headless-new,173,0,199,129,110
edge Ubuntu 22.04 121 selenium headless-new,258,199,0,218,198
firefox Ubuntu 22.04 122 selenium headless,192,129,218,0,89
safari macOS 14.3 17.3 selenium real,173,110,198,89,0


In [23]:
# How many rows exist for tree creation
df[["test_name", "label", "relation_info"]].value_counts()

test_name                   label     relation_info 
framing_iframe              XFO       direct            46840
                                      nested            46840
fullscreen_iframe           PP        child_allow       40720
                                      direct            40720
subresourceloadingCORP_img  CORP      direct            34450
                                                        ...  
framing_object              CSPvsXFO  sandbox             560
fetch_GET                   CORS      simple              480
                                      credentials         480
                                      custom_headers      480
fetch_TEST                  CORS      custom_method       480
Name: count, Length: 73, dtype: int64

In [24]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()


# Limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].transform("nunique") != 1
tree_df = df.loc[condition]
print("Remove tests that are the same in all browsers", len(tree_df))
base_dir = f"trees/{datetime.today().strftime('%Y-%m-%dT%H:%M')}" 

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]
    group = group[["outcome_str", *pred_cols]]
    tree = make_tree(group, pred_cols, group_name, base_dir)
    # print(tree)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,33 days 16 hours 38 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 4 days
H2O_cluster_name:,H2O_from_python_ubuntu_jovqdg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.6 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Remove tests that are the same in all browsers 43260
('accesswindow_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,33 days 16 hours 38 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 4 days
H2O_cluster_name:,H2O_from_python_ubuntu_jovqdg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.6 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('accesswindow_direct', 'direct'), datapoints: 340
('fetch_GET', 'credentials')
Connecting to H2O server at http://localhost:54321 ... successful.


...

Create tree: ('subresourceloadingCORP_object', 'direct'), datapoints: 880
('upgradeHSTS_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,33 days 16 hours 39 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 4 days
H2O_cluster_name:,H2O_from_python_ubuntu_jovqdg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.5 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_direct', 'direct'), datapoints: 2455
('upgradeHSTS_subdomain', 'subdomain')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,33 days 16 hours 39 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 4 days
H2O_cluster_name:,H2O_from_python_ubuntu_jovqdg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.5 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_subdomain', 'subdomain'), datapoints: 2090


# Manual analysis

In [29]:
# Chromium-based treats host-source as case sensitive even though it should not? (URLs and CSP is case-insensitive!)
# Play around: https://observer.sectec.rocks/opg/iframe/?url=https://echo.sectec.rocks/echo/?content-security-policy=img-src%20HTTPS://echo.sectec.rocks&ecocnt_css=%3Cimg%20src=https://echo.sectec.rocks/%3E&content-type=text/html
# Issue is in the scheme match which should be case-insensitive but is in Chromium-based browsers https://w3c.github.io/webappsec-csp/#match-schemes
# host-part matching was fixed here https://issues.chromium.org/issues/41412675, seems like they forgot fixing scheme-sensitivity 
df.loc[df["raw_header"].str.contains("img-src HTTPS://")][["clean_url", "raw_header", "outcome_str", "browser"]].value_counts().to_frame()

count
clean_url                                          raw_header                                         outcome_str browser                                                  
http://sub.headers.websec.saarland/_hp/tests/su... [['content-security-policy', 'img-src HTTPS://S... error       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                                                                                                  chrome Ubuntu 22.04 121 selenium headless-new           1
                                                                                                                  edge Ubuntu 22.04 121 selenium headless-new             1
                                                                                                                  firefox Ubuntu 22.04 122 selenium headless              1
                                                                                                                  safari macOS 14.3 17.3 selenium real                    1
https://sub.headers.websec.saarland/_hp/tests/s... [['content-security-policy', 'img-src HTTPS://S... error       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                                                                                                  chrome Ubuntu 22.04 121 selenium headless-new           1
                                                                                                                  edge Ubuntu 22.04 121 selenium headless-new             1
                                                                                                      load        firefox Ubuntu 22.04 122 selenium headless              1
                                                                                                                  safari macOS 14.3 17.3 selenium real                    1